<a href="https://colab.research.google.com/github/khadeom/Relu/blob/main/web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import pandas as pd
import random
import time
import json


amz_product =[]
data = pd.read_csv('/content/Amazon Scraping - Sheet1.csv')
# print(data.head)
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0','Accept-Language': 'en-US, en;q=0.5'}
for i, j in zip(data['Asin'] ,data['country']):
  product={}
  i=i.strip()
  if i[-1] == "X":

    print(f"URL https://www.amazon.{j}/dp/{i}")

    time.sleep(0.5 *4)
    webpage = requests.get(f"https://www.amazon.{j}/dp/{i}", headers=headers) 
    soup = BeautifulSoup(webpage.content,'lxml')
    #ProductTitle
    try:
      title  = soup.find("span",attrs ={'id':'productTitle'})

      title_value = title.string
      title_string = title_value.strip().replace(',','')
    except:
      title_string ="NA"
    print("product Title = ", title_string)
    product["Product Title"]  =title_string
    # k=json.dumps([title_string])
    #Product Image
    try:
      div=soup.find('div',{"id":"img-canvas"})
      # print(div.find('img').attrs['src'])
      img_url=div.find('img').attrs['src']
    except AttributeError:
      img_url = "NA"
      # print("NA")
    # k=json.dumps([img_url])

    product["Product Image URL"] = img_url
    print("Product Image URL",img_url)
    #Product Price
    try:
      price = soup.find("span", attrs ={'id': 'price'}).string.strip()

    except AttributeError:
      price = "NA"
    k=json.dumps([price])

    # print("Price is :",price)
    div = soup.find('div',{"class":"tmmSwatches"})
    if price =="NA":
      try:
        price = div.findAll("span", {"class": "a-color-base"})
        # print(price)
        if j=="de":
          for i in price:
            if i.text!=None:
              price=i.text.strip()
              break
        elif j =="fr":
          price=price[-1].text.strip()

      except Exception as e:
        price = "NA"
    # print("Price from second method is",price)
    k=json.dumps([price])

    
    if price =="NA":

      try:
        price = div.find("span", {"class": "a-size-base a-color-price a-color-price"})
        # print(price.string)
        price=price.text.strip()
        # print(2,price)
      except AttributeError:
        price = "NA"
    k=json.dumps([price])
    

    product["Price of the Product"] = price
    # j= json.dumps(product)
    print("price is",price)

    #Product Discription
    try:

      div = soup.find('div',{"class":"a-expander-content a-expander-partial-collapse-content"})
      productdetails =div.find("span").string.strip()
      
      
    except AttributeError:
      productdetails="NA"
    product["Product DetailsL"] = productdetails
    print(productdetails)

    amz_product.append(product)


#saving to json file
with open("Product_Info.json", "w") as outfile:
    json.dump(amz_product, outfile)









    